<h1>Paris restaurant analysis</h1>

<h2>Business Problem Description</h2>

<p>France is definitively the country of food and drinks</p>
<p>Having lived in Paris for a big part of my life, I somehow realised that each neighborhood have a singularity regarding food. Each neighborhood has a diverse range of cuisine from all around the world, but it seems that each neighborhood is more keen to have a particular type of cuisine.</p>
<p>Thinking about this, it raises some interesting questions for someone that wants to open a food business:</p>
<ul>
    <li>What are the more and less common specialities of food in the city of Paris ?</li>
    <li>What is each neighborhood food "speciality" and why ? Can we find some clusters and patterns ?</li>
    <li>When opening a restaurant, in which area should we open this new business depending on the type of food ? Should we open this restaurant in an area that contains a lot of this particular food scpeciality or in contrary open it in an area that has a shortage of this particular food kind ?</li>
</ul>
<p>We are going to look for answers to these questions in this analysis</p>
<p>Having answers to these questions can be trully helpfull for new investors that want to open their first restaurant in Paris to make their investment successfull and optimizing profitability of their future businesses.</p>


<h2>Our data</h2>

<p>The city of Paris is basically segmented into 20 districts that each has a postal code going from 75001 to 75020. We are going to use this districts to define our neighborhoods.</p>

<ul>
    <li>The coordinates of each district will be obtained using the <b>pgeocode</b> library, which is easy to use and return coordinates of the lacation with a postal code as input</li>
    <li>The information about restaurants will be obtained with the Foursquare API and loaded in a pandas dataframe</li>
    <li>For each postal code we will be getting the list of restaurants in a 1km radius and conduct analysis on this data to define the food particularities of each district. We will as well define clusters and try to find patterns in this data</li> 
</ul>